In [198]:
import random
from config import DK_USERNAME,DK_PASSWORD

In [200]:
import os
import time
from selenium import webdriver
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : os.getcwd()}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(chrome_options=chromeOptions)  # Optional argument, if not specified will search path.

In [201]:
driver.get("http://www.draftkings.com")

In [202]:
sign_in = driver.find_element_by_link_text('SIGN IN')
sign_in.click()

In [203]:
driver.find_element_by_name('username').send_keys(DK_USERNAME)

In [204]:
driver.find_element_by_name('username').send_keys("\t"+DK_PASSWORD)

In [167]:
driver.find_elements_by_xpath("//*[contains(text(), 'LOG IN')]")[0].click()

In [170]:
driver.get("https://www.draftkings.com/lobby#/NFL")

In [171]:
driver.find_elements_by_xpath("//*[contains(text(), 'View Live Contests')]")[0].click()

In [172]:
prize_pool=driver.find_elements_by_css_selector("div.ui-state-default.slick-header-column.grid-text-with-icon")[2]
prize_pool.click()
prize_pool.click()

In [173]:
from bs4 import BeautifulSoup as soup
import pandas as pd

In [174]:
table=soup(driver.find_element_by_id("lobby-grid").get_attribute("innerHTML"),"lxml")

In [195]:
old_contests=pd.read_csv(file_name)
full_contests=pd.concat([old_contests,live_contests]).drop_duplicates(subset=["link"])
full_contests

,Entries,Fee,Payout,Time,Unnamed: 0,link,name,time_pulled
0,15.1K/17.6K,$4,"$60,000",LIVE!,0.0,/contest/gamecenter/43299739,NFL $60K Front Four (Preseason),2017-08-11 22:49:35.450216
1,7058/7058,$0.25,"$1,500",LIVE!,1.0,/contest/gamecenter/43293788,NFL $1.5K Quarter Jukebox [Just $0.25!] (Prese...,2017-08-11 22:49:35.450216
2,4705/4705,$1,"$4,000",LIVE!,2.0,/contest/gamecenter/43293787,NFL $4K First Down (Preseason),2017-08-11 22:49:35.450216
3,2370/2823,$0.25,200 Tickets,LIVE!,3.0,/contest/gamecenter/43293832,NFL $2M Play-Action SUPERSatellite [200x] (Pre...,2017-08-11 22:49:35.450216
4,1882/1882,$0.25,20 Tickets,LIVE!,4.0,/contest/gamecenter/43293830,NFL $5.55M Fantasy Football Millionaire SUPERS...,2017-08-11 22:49:35.450216
5,1176/1176,$3,"$3,000",LIVE!,5.0,/contest/gamecenter/43299737,NFL $3K Play-Action (Preseason),2017-08-11 22:49:35.450216
6,976/1058,$1,300 Tickets,LIVE!,6.0,/contest/gamecenter/43293831,NFL $2M Play-Action SUPERSatellite [300x] (Pre...,2017-08-11 22:49:35.450216
7,882/882,$1,$750,LIVE!,7.0,/contest/gamecenter/43313515,NFL $750 First Down (Preseason),2017-08-11 22:49:35.450216
8,784/784,$3,"$2,000",LIVE!,8.0,/contest/gamecenter/43309028,NFL $2K Play-Action (Preseason),2017-08-11 22:49:35.450216
9,700/700,$33,"$20,000",LIVE!,9.0,/contest/gamecenter/43299740,NFL $20K Bootleg (Preseason),2017-08-11 22:49:35.450216


In [192]:
file_name="NFL_contests.csv"
try:
	old_contests=pd.read_csv(file_name)
	full_contests=pd.concat([old_contests,live_contests]).drop_duplicates(how=["link"])
except:
	print("No file %s found. Creating new."%file_name)
	full_contests=live_contests

No file NFL_contests.csv found. Creating new.


In [175]:
rows=[]
for row in table.find("div","grid-canvas").find_all("div","slick-row"):
    row_data={}
    for cell in row.find_all("div"):
        if cell.find("a"):
            a=cell.find("a")
            if a.get("data-tracking"):
                row_data[a.get("data-tracking")]=cell.text
            if cell.text=="Watch":
                row_data['link']=a.get("href")
    rows.append(row_data)
live_contests=pd.DataFrame(rows)#missing some elements that you need to scroll for

In [176]:
live_contests

,Entries,Fee,Payout,Time,link,name
0,15.1K/17.6K,$4,"$60,000",LIVE!,/contest/gamecenter/43299739,NFL $60K Front Four (Preseason)
1,7058/7058,$0.25,"$1,500",LIVE!,/contest/gamecenter/43293788,NFL $1.5K Quarter Jukebox [Just $0.25!] (Prese...
2,4705/4705,$1,"$4,000",LIVE!,/contest/gamecenter/43293787,NFL $4K First Down (Preseason)
3,2370/2823,$0.25,200 Tickets,LIVE!,/contest/gamecenter/43293832,NFL $2M Play-Action SUPERSatellite [200x] (Pre...
4,1882/1882,$0.25,20 Tickets,LIVE!,/contest/gamecenter/43293830,NFL $5.55M Fantasy Football Millionaire SUPERS...
5,1176/1176,$3,"$3,000",LIVE!,/contest/gamecenter/43299737,NFL $3K Play-Action (Preseason)
6,976/1058,$1,300 Tickets,LIVE!,/contest/gamecenter/43293831,NFL $2M Play-Action SUPERSatellite [300x] (Pre...
7,882/882,$1,$750,LIVE!,/contest/gamecenter/43313515,NFL $750 First Down (Preseason)
8,784/784,$3,"$2,000",LIVE!,/contest/gamecenter/43309028,NFL $2K Play-Action (Preseason)
9,700/700,$33,"$20,000",LIVE!,/contest/gamecenter/43299740,NFL $20K Bootleg (Preseason)


In [177]:
contest=live_contests.iloc[0]
contest['link']

'/contest/gamecenter/43299739'

In [178]:
BASE_URL="https://www.draftkings.com"

In [179]:
driver.get(BASE_URL+contest['link'])

In [180]:
driver.find_element_by_id("export-lineups-csv").click()

In [46]:
driver.quit()